<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/5_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# 假設你的資料已經載入為 DataFrame `df`
# df = pd.read_excel('your_file.xlsx')  # 如果是從Excel讀取資料

# 判斷 Grade A 判斷條件 (ProGrp ∈ ['H02U-B1', 'H02U-KB', 'H01U-PB'])
def check_grade_a(df):
    df['A_rule'] = 0

    # 條件 a：ProGrp為['H02U-B1', 'H02U-KB', 'H01U-PB']，且param重複 >= 2次（不分fab）
    condition_a = df['ProGrp'].isin(['[H02U-B1]', '[H02U-KB]', '[H01U-PB]'])
    df.loc[condition_a, 'A_rule'] = df.loc[condition_a, 'param'].duplicated(keep=False) & (df.groupby('ProGrp')['param'].transform('count') >= 2)

    # 條件 b：ProGrp 開頭為 '[SF' 或 '[TF'，直接為A
    condition_b = df['ProGrp'].str.startswith('[SF') | df['ProGrp'].str.startswith('[TF')
    df.loc[condition_b, 'A_rule'] = 1

    # 條件 c：其餘 ProGrp，且同fab下param重複 >= 3次，或rule_result中的數值 > 2
    condition_c = ~df['ProGrp'].isin(['[H02U-B1]', '[H02U-KB]', '[H01U-PB]'])
    df.loc[condition_c, 'A_rule'] = (df.groupby(['fab', 'ProGrp'])['param'].transform('count') >= 3) | (df['rule_result'].str.contains(r'\[.*\]') & df['rule_result'].apply(lambda x: any([int(i) > 2 for i in x.strip('[]').split(',')])))

    return df

# 判斷 Grade B 判斷條件 (若該玩家出現>=3次，且monthly_fail_flag=1)
def check_grade_b(df):
    df['B_rule'] = 0

    # 根據ID在當月和前5個月資料中，`monthly_fail_flag == 1` 的次數 >= 3
    fail_count = df[df['monthly_fail_flag'] == 1].groupby('ID').size()
    df['B_rule'] = df['ID'].map(fail_count).ge(3).astype(int)

    return df

# 判斷 Grade C 判斷條件 (如果 A_rule 和 B_rule 都為0，且monthly_fail_flag=1)
def check_grade_c(df):
    df['C_rule'] = 0

    # 當 A_rule 和 B_rule 都是 0，並且 monthly_fail_flag == 1，則為 C
    df['C_rule'] = ((df['A_rule'] == 0) & (df['B_rule'] == 0) & (df['monthly_fail_flag'] == 1)).astype(int)

    return df

# 最終 Grade 判定，依照 A > B > C
def assign_grade(df):
    df['grade'] = ''

    # 優先順序：A > B > C
    df.loc[df['A_rule'] == 1, 'grade'] = 'A'
    df.loc[(df['A_rule'] == 0) & (df['B_rule'] == 1), 'grade'] = 'B'
    df.loc[(df['A_rule'] == 0) & (df['B_rule'] == 0) & (df['C_rule'] == 1), 'grade'] = 'C'

    return df

# 主處理流程
def process_data(df):
    df = check_grade_a(df)  # 判斷 Grade A
    df = check_grade_b(df)  # 判斷 Grade B
    df = check_grade_c(df)  # 判斷 Grade C
    df = assign_grade(df)   # 最終的 Grade 分配
    return df

# 執行主處理流程
df = process_data(df)

# 檢視結果
print(df[['ID', 'A_rule', 'B_rule', 'C_rule', 'grade']])


222

In [ ]:
import pandas as pd

# 假設你的資料 DataFrame 叫 df
def grade_assignment(df):
    # 初始設定新的欄位
    df['A_rule'] = 0
    df['B_rule'] = 0
    df['C_rule'] = 0
    df['grade'] = ''

    # 只對 monthly_fail_flag == 1 的資料進行判斷
    fail_data = df[df['monthly_fail_flag'] == 1]

    # A_rule 判斷
    def check_A_rule(group):
        # 1. ProGrp 條件 a: ProGrp ∈ ['H02U-B1', 'H02U-KB', 'H01U-PB']
        if group['ProGrp'].iloc[0] in ['[H02U-B1]', '[H02U-KB]', '[H01U-PB]']:
            param_check = group['param'].duplicated().sum() >= 1  # param 重複 >= 2次
            rule_result_check = any(group['rule_result'].str.contains(r'\[.*\]', na=False).apply(lambda x: float(x.split('[')[1].split(']')[0]) > 2))  #  rule_result 第一個或第二個大於 2
            if param_check or rule_result_check:
                return True

        # 2. ProGrp 開頭為 [SF 或 [TF
        elif group['ProGrp'].str.startswith(('[SF', '[TF')).any():
            return True

        # 3. 其他 ProGrp：檢查 param 重複 >= 3 次 (分fab)
        else:
            param_check_fab = group.groupby('fab')['param'].apply(lambda x: x.value_counts().max()).max() >= 3  # param 重複 ≥ 3 次
            rule_result_check_fab = any(group['rule_result'].str.contains(r'\[.*\]', na=False).apply(lambda x: float(x.split('[')[1].split(']')[0]) > 2))  # rule_result 大於 2
            if param_check_fab or rule_result_check_fab:
                return True

        return False

    # 遍歷每個玩家 ID (fab + ProGrp + param) 做 A 判斷
    for player_id, group in fail_data.groupby(['fab', 'ProGrp', 'param']):
        if check_A_rule(group):
            df.loc[group.index, 'A_rule'] = 1

    # B_rule 判斷
    def check_B_rule(player_id):
        # 該玩家在 fail 資料中出現 >= 3 次
        return fail_data[fail_data['fab'] == player_id].shape[0] >= 3

    for player_id in fail_data['fab'].unique():
        if check_B_rule(player_id):
            df.loc[df['fab'] == player_id, 'B_rule'] = 1

    # C_rule 判斷
    df.loc[(df['A_rule'] == 0) & (df['B_rule'] == 0) & (df['monthly_fail_flag'] == 1), 'C_rule'] = 1

    # 最終 grade 判斷
    df['grade'] = df.apply(lambda row: 'A' if row['A_rule'] == 1 else ('B' if row['B_rule'] == 1 else ('C' if row['C_rule'] == 1 else '')), axis=1)

    return df

# 呼叫該函數，將結果賦值回原 DataFrame
df = grade_assignment(df)
